In [ ]:
perm = np.random.permutation(X_train.shape[0])
X_train, Y_train = X_train[perm], Y_train[perm]
val_size = int(0.1 * X_train.shape[0])
X_val, Y_val = X_train[:val_size], Y_train[:val_size]
X_train_s, Y_train_s = X_train[val_size:], Y_train[val_size:]

In [3]:
import wandb
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "test_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "epochs": {
            "values": [10, 15, 20]
        },
        "batch_size": {
            "values": [32, 64]
        },
        "learning_rate": {
            "values": [0.01, 0.005, 0.001]
        },
        "optimizer": {
            "values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]
        },
        "hidden_layers": {
            "values": [
                [64],
                [128],
                [128, 64],
                [256, 128]
            ]
        }
    }
}

In [ ]:
def train_wandb():
    run = wandb.init()
    config = wandb.config
    model = NeuralNet(
        input_dim=784,
        hidden_layers=config.hidden_layers,
        output_dim=10,
        optimizer=config.optimizer,
        lr=config.learning_rate
    )
    model.train(
        X_train,
        Y_train,
        X_test,
        Y_test,
        epochs=config.epochs,
        batch_size=config.batch_size
    )
    Y_pred_test, _ = model.forward(X_test)
    test_acc = np.mean(
        np.argmax(Y_pred_test, axis=1) == np.argmax(Y_test, axis=1)
    )
    wandb.log({
        "test_accuracy": test_acc
    } , commit=False)

In [ ]:
sweep_id = wandb.sweep(
    sweep_config,
    project="fashion-mnist-q4"
)
wandb.agent(
    sweep_id,
    function=train_wandb,
    count=20
)